### Tecnolgías del Lenguaje. Entregable 4
---
#### Corrección del archivo CSV original

El dataset original `material/posts.csv` contiene dos columnas: `username` y `body`, donde `body` corresponde al texto completo de cada publicación. Sin embargo, hemos detectado que el archivo presenta inconsistencias en el uso de comillas, ya que algunos posts aparecen entrecomillados y otros no. Esto provoca errores al leer el archivo con `pandas.read_csv()`, ya que el separador de coma se confunde con las comas internas del texto, generando filas con más de dos columnas o registros mal alineados.

Para resolver este problema, se utiliza el módulo estándar `csv` de Python, que permite reescribir el archivo garantizando un formato CSV válido y seguro. El código realiza las siguientes operaciones:
- Lectura del archivo original (`material/posts.csv`) con el lector csv.reader, tolerando posibles errores de codificación.
- Creación de un nuevo archivo corregido (`material/posts_corregido.csv`), utilizando `csv.writer` con la opción `quoting=csv.QUOTE_ALL`. Esta opción fuerza que todas las celdas se escriban entre comillas dobles, escapando automáticamente las comillas internas (`"` → `""`) según el **estándar CSV (RFC 4180)**.
- Reparación de filas mal formateadas: Si una fila tiene más de dos columnas, se asume que el texto del post contenía comas no entrecomilladas;
por tanto, se recombinan todas las columnas a partir de la segunda (`",".join(row[1:])`) para reconstruir el contenido completo del post. Si la fila tiene exactamente dos columnas, se escribe tal cual. Si la fila está vacía o corrupta, se omite.
- Escritura del archivo corregido, que ahora tiene exactamente dos columnas por línea (`username`, `body`), con las comillas y los caracteres especiales correctamente escapados.

El resultado es un archivo estructuralmente coherente, legible por pandas sin errores y que conserva íntegramente el contenido textual original de los posts. Es útil almacenarlo porque puede servir de *checkpoint* en nuevas soluciones.

In [1]:
import csv

input_path = "material/posts.csv"
output_path = "material/posts_corregido.csv"

with open(input_path, "r", encoding="utf-8", errors="replace") as fin, \
     open(output_path, "w", newline="", encoding="utf-8") as fout:
    
    reader = csv.reader(fin)
    writer = csv.writer(fout, quoting=csv.QUOTE_ALL)
    
    for row in reader:
        # Si la línea tiene más de dos columnas, las juntamos
        if len(row) > 2:
            username = row[0]
            body = ",".join(row[1:])  # volver a unir texto roto por comas
            writer.writerow([username, body])
        elif len(row) == 2:
            writer.writerow(row)
        else:
            # si por error hay líneas vacías, las saltamos
            continue

print(f"✅ CSV corregido guardado en {output_path}")

✅ CSV corregido guardado en material/posts_corregido.csv


Algunos posts son demasiado largos y el modelo SBERT no podrá computar toda la entrada simultáneamente. Por ello, decidimos **dividir los posts más largos para adaptarlos al límite de la entrada del modelo**.

In [ ]:
from transformers import AutoTokenizer
from tqdm import tqdm

INPUT_PATH = "material/posts_corregido.csv"
OUTPUT_PATH = "material/posts_corregido_dividido.csv""

MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# Detectar límites
MODEL_MAX = tokenizer.model_max_length
DESIRED_MAX = 256
try:
    special = tokenizer.num_special_tokens_to_add(pair=False)
except Exception:
    special = 2

CHUNK_SIZE = min(DESIRED_MAX, MODEL_MAX - special)
OVERLAP = 50


def split_text_on_tokens_safe(text, chunk_size=CHUNK_SIZE, overlap=OVERLAP):
    token_ids = tokenizer.encode(text, add_special_tokens=False)
    n = len(token_ids)
    if n <= chunk_size:
        return [text.strip()]

    chunks = []
    step = chunk_size - overlap if (0 < overlap < chunk_size) else chunk_size
    if step <= 0:
        step = chunk_size

    for start in range(0, n, step):
        end = start + chunk_size
        chunk_ids = token_ids[start:end]
        chunk_text = tokenizer.decode(
            chunk_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        ).strip()

        if chunk_text:
            chunks.append(chunk_text)
        if end >= n:
            break

    return chunks


# --- PROCESAMIENTO CON BARRA DE PROGRESO ---
# Primero contamos cuántas filas tiene el CSV
with open(INPUT_PATH, "r", encoding="utf-8", errors="replace") as fin:
    total_lines = sum(1 for _ in fin)

with open(INPUT_PATH, "r", encoding="utf-8", errors="replace") as fin, \
     open(OUTPUT_PATH, "w", newline="", encoding="utf-8") as fout:

    reader = csv.reader(fin)
    writer = csv.writer(fout, quoting=csv.QUOTE_ALL)

    writer.writerow(["username", "fragment_index", "fragment_text"])

    for row in tqdm(reader, total=total_lines, desc="Procesando"):
        if not row:
            continue

        if len(row) > 2:
            username = row[0]
            body = ",".join(row[1:])
        elif len(row) == 2:
            username, body = row
        else:
            continue

        fragments = split_text_on_tokens_safe(body)

        for i, frag in enumerate(fragments):
            writer.writerow([username, i, frag])

print("✅ Listo. Archivo generado en:", OUTPUT_PATH)

Nuestro objetivo final es realizar predicciones sobre los valores Big-5 (openness, conscientiousness, extraversion, agreeableness, neuroticism) **para un autor (username)** de posts. Por tanto, debemos agrupar los posts por usuario.

En este paso preparamos el dataset para análisis de similitud semántica: **cada fila corresponde a un usuario único** y **la columna body contiene una lista de strings, cada string es un post de ese usuario**. Para ello:
- Cargamos el CSV corregido que acabamos de crear, `posts_corregido.csv`.
- Agrupamos los posts por `username`.
- Limpiamos posibles valores NaN dentro de las listas de posts.
- Verificamos que la estructura es correcta:
    - Columnas: `username`, `body`, donde cada `body` es una lista de strings.
- Guardamos el CSV final post_by_author.csv listo para procesamiento con modelos de NLP.

In [2]:
import pandas as pd
import math

input_path = "material/posts_corregido_dividido.csv"
output_path = "material/posts_by_author.csv"

# Cargar CSV corregido
df = pd.read_csv(input_path)

# Agrupar posts por usuario y convertir a lista de strings
grouped = df.groupby('username')['body'].apply(list).reset_index()

# Limpiar posibles NaN dentro de las listas de posts
grouped['body'] = grouped['body'].apply(lambda posts: [p for p in posts if isinstance(p, str) and not (isinstance(p, float) and math.isnan(p))])

# Verificación rápida
print("Columnas:", grouped.columns.tolist())
print("Primeros registros:\n", grouped.head())
print("Tipo 'body' de la primera fila:", type(grouped.loc[0, 'body']))
print("Primer post de la primera fila:", grouped.loc[0, 'body'][0])

# Comprobar que 'username' es string
all_usernames_str = all(isinstance(u, str) for u in grouped['username'])
print("Todos los usernames son cadenas de texto?:", all_usernames_str)

# Comprobar que 'body' es lista de strings
all_bodies_correct = True
for i, row in grouped.iterrows():
    body = row['body']
    # Pandas lee listas de CSV como strings, así que primero evaluamos a lista real
    try:
        posts = eval(body) if isinstance(body, str) else body
    except:
        posts = []
    if not isinstance(posts, list) or not all(isinstance(p, str) for p in posts):
        all_bodies_correct = False
        print(f"Fila {i} incorrecta:")
        print("username:", row['username'])
        print("body:", body[:200])  # mostrar los primeros 200 caracteres
        break

print("Todas las filas de 'body' son listas de strings?:", all_bodies_correct)

# Guardar CSV
grouped.to_csv(output_path, index=False, quoting=1)  # quoting=csv.QUOTE_ALL

print(f"✅ Dataset agrupado por usuario guardado en {output_path}")

Columnas: ['username', 'body']
Primeros registros:
        username                                               body
0   -Areopagan-  [Your first and second question is the same qu...
1     -BigSexy-  [I've been asked to cum everywhere with my ex ...
2    -BlitzN9ne  [I'm currently in the middle of making a Payda...
3  -CrestiaBell  [First and foremost I extend my condolences to...
4        -dyad-  [I failed both...I'm great at reading people i...
Tipo 'body' de la primera fila: <class 'list'>
Primer post de la primera fila: Your first and second question is the same question. I'll try to make it more incisive for you because you don't articulate what you want to know. Do I think people who cooperate also compete? Yeah, sure, obviously, in a separate way people who play chess, or style themselves in order to be attractive, merely agree that there is a game and the game has rules. Fine. I don't think they cooperate in terms of winning the game because if I know, especially because you